In [1]:
import numpy as np
import pandas as pd
import holoviews as hv
import panel as pn
import folium
import json
import matplotlib.cm as cm  # Import colormap library
import geopandas as gpd
import branca.colormap as cm
from colorcet import bmy

pn.extension('tabulator', template='fast')
import hvplot.pandas

## Create intro

In [2]:
introduction = pn.pane.Markdown("""
<h1>GeoPostal Insight</h1>
UPU Innovation Challenge 2024
""", width=600)

panel_logo = pn.pane.PNG(
    'https://panel.holoviz.org/_static/logo_stacked.png',
    link_url='https://panel.holoviz.org', height=95, align='center'
)

event_logo = pn.pane.PNG(
    'https://upload.wikimedia.org/wikipedia/commons/2/2d/Universal_Postal_Union_logo.svg',
    link_url='https://dribdat.hackathon.post/event/3', height=95, align='center'
)

itu_logo = pn.pane.PNG(
    'https://upload.wikimedia.org/wikipedia/commons/e/e1/International_Telecommunication_Union_logo.svg',
    link_url='https://dribdat.hackathon.post/event/3', height=95, align='center'
)

intro = pn.Row(
    event_logo,
    itu_logo,
    introduction,
    pn.layout.HSpacer(),
    panel_logo,
    sizing_mode='stretch_width'
)

intro

BokehModel(combine_events=True, render_bundle={'docs_json': {'51ed2580-27ec-4534-96d2-f03bef7c0ed4': {'version…

### Load and cache data

In [3]:
from holoviews.element.tiles import lon_lat_to_easting_northing

@pn.cache
def load_data():
    df = pd.read_csv('data/brazil_geodata_h3_res4.csv')
    df['x'], df['y'] = lon_lat_to_easting_northing(df['X'], df['Y'])
    return df

df = load_data()

df.tail()

,X,Y,index,h3_resolution,post_offices_count,postal_bank_count,road_length_km,cell_towers_count,population_count,x,y
4490,-49.707909,-12.739338,84a8d93ffffffff,4,0,0,1541.716087,9.0,2440.051854,-5.533459e+06,-1.429968e+06
4491,-51.197174,-4.880746,84814b1ffffffff,4,0,0,137.846629,NaN,197.881241,-5.699243e+06,-5.439805e+05
4492,-47.718735,-4.314751,84815b5ffffffff,4,0,1,3178.472903,165.0,60535.689792,-5.312025e+06,-4.807705e+05
4493,-49.516998,-1.504511,848044bffffffff,4,0,1,59.232572,50.0,41480.516639,-5.512207e+06,-1.675007e+05
4494,-51.731358,-16.748201,84a8ea9ffffffff,4,0,1,1844.737272,19.0,4230.374824,-5.758708e+06,-1.891534e+06


### Set up linked selections

In [4]:
ls = hv.link_selections.instance()

def clear_selections(event):
    ls.selection_expr = None

clear_button = pn.widgets.Button(name='Clear', align='center')

clear_button.param.watch(clear_selections, 'clicks');

total_area = df.road_length_km.sum()

def count(data):
    selected_area  = np.sum(data['road_length_km'])
    selected_percentage = selected_area / total_area * 100
    return f'## Roads: {len(data)} | Selected: {selected_area:.0f} km ({selected_percentage:.1f}%)</font>'

pn.Row(
    pn.pane.Markdown(pn.bind(count, ls.selection_param(df)), align='center', width=600),
    clear_button
).servable(area='header', title='GeoPostal Insight')

BokehModel(combine_events=True, render_bundle={'docs_json': {'482680ea-08bd-4da0-ab9e-8dc88d285a42': {'version…

In [5]:
instruction = pn.pane.Markdown("""
💡 This dashboard visualizes postal offices, allows exploring the relationships between regional characteristics such as transport and telco infrastructures, complementary public services. Use Box-select on each plot to subselect and hit the "Clear" button to reset. Here you can ask prompts about our model, for example:
<ul><li>Create a report about the state of postal sector in Manaus region of Brazil.</li>
<li>What is the Integrated Index for Postal Development?</li></ul>
""", width=700)

# Todo: replace dynamically with output of prompt
hugging_pane = pn.pane.HTML("""
<iframe src="https://hf.co/chat/assistant/6659aa1af82f39d447cd13f3" width="100%" height="460" frameborder="0"></iframe>
""", width=700)

## Rich mapping

In [6]:
# Load the GeoJSON data
geojson_data = "data/brazil_geodata_h3_res4.geojson"
gdf = gpd.read_file(geojson_data)

# Calculate the population count deciles for color mapping
population_counts = gdf['population_count']
population_deciles = pd.qcut(population_counts, 10, labels=False, duplicates='drop')

# Create a dictionary to map each population count to its decile
pop_count_to_decile = dict(zip(population_counts, population_deciles))

# Create a linear colormap
colormap = cm.linear.YlOrRd_09.scale(0, 9)
colormap = colormap.to_step(10)  # Use 10 steps for the deciles

# Define a function to style the features
def style_function(feature):
    population_count = feature['properties']['population_count']
    decile = pop_count_to_decile[population_count]
    return {
        'fillOpacity': 0.7,
        'weight': 0,
        'fillColor': colormap(decile)
    }

# Create the map
m = folium.Map(location=[-9.29, -51.81], zoom_start=3, tiles="cartodb positron")
folium_pane = pn.pane.plot.Folium(m, height=460)
folium.Marker(
    [-9.29, -51.81], popup="<i>Brazil</i>", tooltip="Work in progress!"
).add_to(m)

# Add the GeoJSON data to the map with the style function
folium.GeoJson(
    geojson_data,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['population_count']),
    name="GeoPostal Impact"
).add_to(m)

# Add a layer control
folium.LayerControl().add_to(m)

# Set the pane object to the map
folium_pane.object = m

# Create the dashboard layout
h3ai = pn.Row(
    folium_pane,
    hugging_pane
)
h3ai

BokehModel(combine_events=True, render_bundle={'docs_json': {'520c06dd-a1b1-431b-8743-ef3f87905f07': {'version…

### Create plots

In [7]:
df['post_offices_count_int'] = df['post_offices_count'].fillna(0).astype(int)
df['postal_bank_count_int'] = df['postal_bank_count'].fillna(0).astype(int)

geo = df.hvplot.points(
    'x', 'y', rasterize=True, tools=['hover'], tiles='OSM', cmap=bmy, logz=False, colorbar=True,
    xaxis=None, yaxis=False, min_height=400, responsive=True
).opts('Tiles', alpha=0.8)

scatter1 = df.hvplot.scatter(
    'post_offices_count', 'population_count', 
    xlabel='Post offices', ylabel='Population', color='postal_bank_count_int', responsive=True, min_height=400
)

scatter2 = df.hvplot.scatter(
    'road_length_km', 'post_offices_count', 
    xlabel='Road length', ylabel='Post offices', responsive=True, min_height=400
)

scatter3 = df.hvplot.scatter(
    'cell_towers_count', 'post_offices_count', 
    xlabel='Cell towers', ylabel='Post offices', responsive=True, min_height=400
)

hist1 = df.hvplot.hist(
    'post_offices_count_int', xlabel='Number of Post offices',
    fontscale=1.2, responsive=True, min_height=350, fill_color='#85c1e9'
)


hist2 = df.hvplot.hist(
    'postal_bank_count_int', xlabel='Number of Postal banks',
    fontscale=1.2, responsive=True, min_height=350, fill_color='#f1948a'
)

plots = pn.pane.HoloViews(ls(geo + scatter1 + hist1 + hist2 + scatter2 + scatter3).cols(2).opts(sizing_mode='stretch_both'))
plots

/anaconda/envs/giga-inframapkit-env/lib/python3.9/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


BokehModel(combine_events=True, render_bundle={'docs_json': {'2318dab4-e9e9-45e2-9198-6ee97b269b4f': {'version…

## Dashboard content


In [8]:
pn.Column(intro, instruction, h3ai, plots, sizing_mode='stretch_both').servable()


BokehModel(combine_events=True, render_bundle={'docs_json': {'2722f82c-8bf4-4a42-b682-cf51269d9945': {'version…

Task exception was never retrieved
future: <Task finished name='Task-7' coro=<Callback.process_on_change() done, defined at /anaconda/envs/giga-inframapkit-env/lib/python3.9/site-packages/holoviews/plotting/bokeh/callbacks.py:356> exception=UnsetValueError("figure(id='026f3285-9122-4136-9c2b-37c5de562891', ...).inner_height doesn't have a value set")>
Traceback (most recent call last):
  File "/anaconda/envs/giga-inframapkit-env/lib/python3.9/site-packages/holoviews/plotting/bokeh/callbacks.py", line 375, in process_on_change
    msg[attr] = self.resolve_attr_spec(path, cb_obj)
  File "/anaconda/envs/giga-inframapkit-env/lib/python3.9/site-packages/holoviews/plotting/bokeh/callbacks.py", line 282, in resolve_attr_spec
    resolved = getattr(resolved, p, None)
  File "/anaconda/envs/giga-inframapkit-env/lib/python3.9/site-packages/bokeh/core/property/descriptors.py", line 283, in __get__
    raise UnsetValueError(f"{obj}.{self.name} doesn't have a value set")
bokeh.core.property.descrip

Task exception was never retrieved
future: <Task finished name='Task-12' coro=<Callback.process_on_change() done, defined at /anaconda/envs/giga-inframapkit-env/lib/python3.9/site-packages/holoviews/plotting/bokeh/callbacks.py:356> exception=UnsetValueError("figure(id='1a0e2daf-5bf0-4bd8-b431-94815583548d', ...).inner_height doesn't have a value set")>
Traceback (most recent call last):
  File "/anaconda/envs/giga-inframapkit-env/lib/python3.9/site-packages/holoviews/plotting/bokeh/callbacks.py", line 375, in process_on_change
    msg[attr] = self.resolve_attr_spec(path, cb_obj)
  File "/anaconda/envs/giga-inframapkit-env/lib/python3.9/site-packages/holoviews/plotting/bokeh/callbacks.py", line 282, in resolve_attr_spec
    resolved = getattr(resolved, p, None)
  File "/anaconda/envs/giga-inframapkit-env/lib/python3.9/site-packages/bokeh/core/property/descriptors.py", line 283, in __get__
    raise UnsetValueError(f"{obj}.{self.name} doesn't have a value set")
bokeh.core.property.descri